In [ ]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
import seaborn as sns

# Loading Files

In [ ]:
my_path = '/Users/deagogishvili/Desktop/PublicationFiles/ALL_FILES_NEEDED'

nsp_uniprot = my_path + '/uniprot_9606_formatted_lisa.tab'
all_uniprot = my_path + '/uniprot_9606.tab'
lhpsa = my_path + '/lhpsa_netsurfp_model_predictions.tsv'
TM_uniprot = my_path + '/TM_proteins.tab'
consensus = my_path + '/rna_consensus.tsv'
lp_all = my_path + '/ready_to_use_data.csv'

nsp_uniprot = pd.read_csv(nsp_uniprot, sep='\t', engine='python') #NSP2 predictions for Uniprot
lhpsa = pd.read_csv(lhpsa, sep='\t', engine='python') # LHP predictions
all_uniprot = pd.read_csv(all_uniprot, sep='\t', engine='python') #all information from Uniprot only
TM_uniprot = pd.read_csv(TM_uniprot, sep='\t', engine='python') #annotated transmembrane proteins
consensus = pd.read_csv(consensus, sep='\t', engine='python') #mRNA expression consensus data from HPA
lp_all = pd.read_csv(lp_all, sep=',', engine='python') #Structure-based definitions from Molpatch

# Removing Annotated Transmembrane Proteins from Uniprot

In [ ]:
TM_uniprot = list(TM_uniprot['Entry'])
all_uniprot_without_tm = all_uniprot[~all_uniprot['Entry'].isin(TM_uniprot)]

# Data Curation

In [ ]:
print('all', len(nsp_uniprot.iloc[:,0]))

# Remove disordered proteins
nsp_uniprot = nsp_uniprot[nsp_uniprot['disorder'] < 0.5]
print('< 0.5 Disorder', len(nsp_uniprot.iloc[:,0]))

# Remove Large proteins
#nsp_uniprot['length'] = nsp_uniprot['length'].astype(float)
nsp_uniprot = nsp_uniprot[nsp_uniprot['length'] <= 800]
print('Short Proteins', len(nsp_uniprot.iloc[:,0]))

# remove ambiguous predictions
nsp_uniprot = nsp_uniprot[nsp_uniprot['tasa_netsurfp2'] > 0]
nsp_uniprot = nsp_uniprot[nsp_uniprot['thsa_netsurfp2'] > 0]
nsp_uniprot = nsp_uniprot[nsp_uniprot['rhsa_netsurfp2'] > 0]
nsp_uniprot = nsp_uniprot[nsp_uniprot['rhsa_netsurfp2'] < 1]
print('TASA, THSA > 0 and 0 < RHSA < 1', len(nsp_uniprot.iloc[:,0]))

In [ ]:
all_uniprot.columns = ['id', 'x1', 'x2', 'x3', 'Gene_ID', 'x4', 'x5', 'x6', 'length']
all_uniprot = all_uniprot[['id', 'Gene_ID','length']]
all_uniprot = all_uniprot.dropna(subset = ['Gene_ID'])

hg = nsp_uniprot[['id', 'thsa_netsurfp2', 'rhsa_netsurfp2']] # hg = human genome
hg = pd.merge(hg, all_uniprot, on='id')
hg = pd.merge(hg, lhpsa, on='id')
hg = hg.rename(columns = {'id': 'Uniprot_ID', 'thsa_netsurfp2': 'THSA', 'rhsa_netsurfp2':'RHSA', 'prediction':'LHPSA'}, inplace = False)
hg = hg[['Uniprot_ID', 'Gene_ID', 'THSA', 'RHSA', 'LHPSA', 'length']]

# discart multiple Gene_IDs, keep only the first one
hg['Gene_ID'] = hg['Gene_ID'].astype(str)
hg['Gene_ID'] = [x.split(';')[0] for x in hg['Gene_ID']]

# Discard duplicate Gene_ID, keep the one with the highest THSA
hg = hg.sort_values('THSA').drop_duplicates(subset=['Gene_ID'], keep='last') 
print(len(hg.iloc[:,0]))

# Remove NaNs from Gene names
hg = hg.dropna(subset = ['Gene_ID'])
print(len(hg.iloc[:,0]))

# Data Curation without Transmembrane proteins in it

In [ ]:
all_uniprot_without_tm.columns = ['id', 'x1', 'x2', 'x3', 'Gene_ID', 'x4', 'x5', 'x6', 'length']
all_uniprot_without_tm = all_uniprot_without_tm[['id', 'Gene_ID', 'length']]

# hg (human genome) without transmembrane proteins
hg_without_tm = nsp_uniprot[['id', 'thsa_netsurfp2', 'rhsa_netsurfp2']]
hg_without_tm = pd.merge(all_uniprot_without_tm, hg_without_tm, on='id')
hg_without_tm = pd.merge(hg_without_tm, lhpsa, on='id')
hg_without_tm = hg_without_tm.rename(columns = {'id': 'Uniprot_ID', 'thsa_netsurfp2': 'THSA', 'rhsa_netsurfp2':'RHSA', 'prediction':'LHPSA'}, inplace = False)
hg_without_tm = hg_without_tm[['Uniprot_ID', 'Gene_ID', 'THSA', 'RHSA', 'LHPSA', 'length']]

# discart multiple Gene_IDs, keep only the first one

hg_without_tm['Gene_ID'] = hg_without_tm['Gene_ID'].astype(str)
hg_without_tm['Gene_ID'] = [x.split(';')[0] for x in hg_without_tm['Gene_ID']]

# Discard duplicate Gene_ID, keep the one with the highest THSA

hg_without_tm = hg_without_tm.sort_values('THSA').drop_duplicates(subset=['Gene_ID'], keep='last') 
print(len(hg_without_tm.iloc[:,0]))

# Plotting Distribution Figures

In [ ]:
sns.set_style('ticks')
sns.set(font_scale = 1.55)
fig, ax = plt.subplots(0,0)
fig.set_size_inches(14, 5)

sns.distplot(lp_all['thsa'], hist=False, kde=True, color='Blue', kde_kws={'shade': True, 'linewidth': 2})
sns.distplot(hg['THSA'], hist=False, kde=True, color='crimson', kde_kws={'shade': True, 'linewidth': 2})
sns.distplot(hg_without_tm['THSA'], hist=False, kde=True, color='goldenrod', kde_kws={'shade': True, 'linewidth': 2})
#legend = ['Human proteome','Structure-based set']
#plt.legend(legend, prop={'size': 15}, title = 'Data Set')
#plt.title('',  size=30)
plt.xlabel('THSA ($Å^2$)',  size=25)
plt.ylabel('Density',  size=25)
#fig.savefig(my_path + 'THSA.png', dpi = 150)

# Below is the code for RHSA, LHP and Protein length

fig, ax = plt.subplots(0,0)
fig.set_size_inches(14, 5)
sns.distplot(hg['RHSA'], hist=False, kde=True, color='crimson', kde_kws={'shade': True, 'linewidth': 2})
sns.distplot(lp_all['rhsa'], hist=False, kde=True, color='Blue', kde_kws={'shade': True, 'linewidth': 2})
sns.distplot(hg_without_tm['RHSA'], hist=False, kde=True, color='goldenrod', kde_kws={'shade': True, 'linewidth': 2})
plt.xlabel('RHSA',  size=25)
plt.ylabel('Density',  size=25)

fig, ax = plt.subplots(0,0)
fig.set_size_inches(14, 5)
sns.distplot(hg['LHPSA'], hist=False, kde=True, color='crimson', kde_kws={'shade': True, 'linewidth': 2})
sns.distplot(lp_all['size'], hist=False, kde=True, color='Blue', kde_kws={'shade': True, 'linewidth': 2})
sns.distplot(hg_without_tm['LHPSA'], hist=False, kde=True, color='goldenrod', kde_kws={'shade': True, 'linewidth': 2})
plt.xlabel('LHP ($Å^2$)',  size=25)
plt.ylabel('Density',  size=25)

fig, ax = plt.subplots(0,0)
fig.set_size_inches(15, 5)
sns.distplot(hg['length'], hist=False, kde=True, color='crimson', kde_kws={'shade': True, 'linewidth': 2})
sns.distplot(lp_all['length'], hist=False, kde=True, color='Blue', kde_kws={'shade': True, 'linewidth': 2})
sns.distplot(hg_without_tm['length'], hist=False, kde=True, color='goldenrod',kde_kws={'shade': True, 'linewidth': 2})

plt.xlabel('Protein Length',  size=25)
plt.ylabel('Density',  size=25)

# THSA, RHSA and LHP Relationship

In [ ]:
#sns.set_style(style='white')
fig , ax = plt.subplots(figsize = (17,8))
sns.set(font_scale = 1.55)

cm = plt.cm.get_cmap('coolwarm')
lp_all = lp_all.sort_values(by=['rhsa'], ascending=True)

x = lp_all['size']
y = lp_all['thsa']
colors = lp_all['rhsa'] 

im = plt.scatter(x, y, s=60, c=colors, alpha=0.5, cmap=cm)
fig.colorbar(im, ax=ax, label='RHSA')
ax.set_xlabel('LHP($Å^2$)', fontsize=20)
ax.set_ylabel('THSA($Å^2$)', fontsize=20)
ax.set_title('Structure-based Data', fontsize=30)
plt.show()

#fig.savefig(my_path + 'THSA_RHSA_LHPSA_Scatter_structure-based.png', dpi = 150)

# The same plot for the human proteome data

fig , ax = plt.subplots(figsize = (17,8))
sns.set(font_scale = 1.55)

cm = plt.cm.get_cmap('coolwarm')
hg = hg.sort_values(by=['RHSA'], ascending=True)

x = hg['LHPSA']
y = hg['THSA']
colors =hg['RHSA'] 

im = plt.scatter(x, y, s=60, c=colors, alpha=0.5, cmap=cm)
fig.colorbar(im, ax=ax, label='RHSA')
ax.set_xlabel('LHP($Å^2$)', fontsize=20)
ax.set_ylabel('THSA($Å^2$)', fontsize=20)
ax.set_title('Human Proteome Predictions', fontsize=30)
plt.show()

# Analysing GSEA on Disease pathways and overlapping proteins with KEGG ND gene sets

In [ ]:
KEGG_PD = my_path + '/KEGG_PARKINSONS_DISEASE.tsv'
KEGG_PD = pd.read_csv(KEGG_PD, sep='\t', engine='python')
KEGG_PD = KEGG_PD.rename(columns = {'SYMBOL': 'Gene_ID'}, inplace = False)
KEGG_PD = pd.merge(hg, KEGG_PD, on='Gene_ID')

KEGG_HD = my_path + '/KEGG_HUNTINGTONS_DISEASE.tsv'
KEGG_HD = pd.read_csv(KEGG_HD, sep='\t', engine='python')
KEGG_HD = KEGG_HD.rename(columns = {'SYMBOL': 'Gene_ID'}, inplace = False)
KEGG_HD = pd.merge(hg, KEGG_HD, on='Gene_ID')

KEGG_AD = my_path + '/KEGG_ALZHEIMERS_DISEASE.tsv'
KEGG_AD = pd.read_csv(KEGG_AD, sep='\t', engine='python')
KEGG_AD = KEGG_AD.rename(columns = {'SYMBOL': 'Gene_ID'}, inplace = False)
KEGG_AD = pd.merge(hg, KEGG_AD, on='Gene_ID')

print('overlapping proteins with KEGG Parkinsons', KEGG_PD['length'].median(), 'amino acid residues')
print('overlapping proteins with KEGG Huntingtons', KEGG_HD['length'].median(), 'amino acid residues')
print('overlapping proteins with KEGG Alzheimers', KEGG_AD['length'].median(), 'amino acid residues')

# mRNA Expression Consensus data

In [ ]:
consensus = consensus.rename(columns = {'Gene name': 'Gene_ID'}, inplace = False)
consensus = pd.merge(hg, consensus, on='Gene_ID')
consensus_wtmp = consensus[~consensus['Uniprot_ID'].isin(TM_uniprot)] # without transmembrane proteins

# Overall Hydrophobicity = Tissue-specific average surface hydrophobicity (TASH)

In [ ]:
def tissue_hydrophobicity(df):
    df['NX_THSA'] = df['NX']*df['THSA']
    df['NX_RHSA'] = df['NX']*df['RHSA']
    df['NX_LHPSA'] = df['NX']*df['LHPSA']

    df['sum_NX']= df.groupby("Tissue")["NX"].transform('sum')
    df['sum_NX_THSA']= df.groupby("Tissue")["NX_THSA"].transform('sum')
    df['sum_NX_RHSA']= df.groupby("Tissue")["NX_RHSA"].transform('sum')
    df['sum_NX_LHPSA']= df.groupby("Tissue")["NX_LHPSA"].transform('sum')

    df['h_THSA'] = df['sum_NX_THSA']/df['sum_NX']
    df['h_RHSA'] = df['sum_NX_RHSA']/df['sum_NX']
    df['h_LHPSA'] = df['sum_NX_LHPSA']/df['sum_NX']
    df = df[['Tissue', 'h_THSA', 'h_RHSA', 'h_LHPSA']]
    df = df.drop_duplicates(subset=['Tissue'])
    
    return(df)

In [ ]:
overall_h_wtmp = tissue_hydrophobicity(consensus_wtmp)
overall_h_wtmp = overall_h_wtmp.sort_values('h_THSA', ascending=False)
overall_h = tissue_hydrophobicity(consensus)
overall_h = overall_h.sort_values('h_THSA', ascending=False)

In [ ]:
# Merge the data sets
overall_h_wtmp.columns = ['Tissue', 'h_THSA_wtmp', 'h_RHSA_wtmp', 'h_LHPSA_wtmp']
TASH = pd.merge(overall_h, overall_h_wtmp, on='Tissue')
TASH.columns = ['Tissue', 'THSA', 'RHSA', 'LHP', 'THSA#', 'RHSA#', 'LHP#']
#TASH.to_csv(my_path + '/TASH.csv', sep= ',', index = False)

In [ ]:
import seaborn as sns
import dataframe_image as dfi

s = TASH.style\
    .background_gradient(cmap='coolwarm', subset=['THSA'])\
    .background_gradient(cmap='coolwarm', subset=['RHSA'])\
    .background_gradient(cmap='coolwarm', subset=['LHP'])\
    .background_gradient(cmap='coolwarm', subset=['THSA#'])\
    .background_gradient(cmap='coolwarm', subset=['RHSA#'])\
    .background_gradient(cmap='coolwarm', subset=['LHP#'])
#dfi.export(s, my_path + 'TASH_table.png')
s

# Relationship between mRNA expression data and THSA, RHSA, LHP values

In [ ]:
consensus = my_path + '/rna_consensus.tsv'
consensus = pd.read_csv(consensus, sep='\t', engine='python') #mRNA expression consensus data from HPA

consensus = consensus.rename(columns = {'Gene name': 'Gene_ID'}, inplace = False)
consensus = pd.merge(hg, consensus, on='Gene_ID')
#consensus_wtmp = consensus[~consensus['Uniprot_ID'].isin(TM_uniprot)] # without transmembrane proteins

# Analysis normalised expression (NX) value

In [ ]:
#calculate median NX value per gene across all the tissues that a gene appears in
consensus['Median_NX'] = consensus.groupby(['Gene_ID'])['NX'].transform('median') 

#keep only one entry per gene by dropping duplicates 
#and keeping also the highest NX value per gene across the tissues
NX_median = consensus.sort_values('NX').drop_duplicates(subset=['Gene_ID'], keep='last') #df for analysing median NX value
NX_highest = consensus.sort_values('NX').drop_duplicates(subset=['Gene_ID'], keep='last') #df for analysing the highest NX value

# Highest NX value

NX_highest['NX'].describe()

In [ ]:
def classifier_highest_NX(df):
    if df['NX'] <= 11.7:
        return 'A'
    elif df['NX'] > 11.7 and df['NX'] <= 21.4:
        return 'B'
    elif df['NX'] > 21.4 and df['NX'] <= 28.6:
        return 'C'
    elif df['NX'] > 28.6 and df['NX'] <= 35:
        return 'D'
    elif df['NX'] > 35 and df['NX'] <= 42.3:
        return 'E'
    elif df['NX'] > 42.3 and df['NX'] <= 51.2:
        return 'F'    
    elif df['NX'] > 51.2 and df['NX'] <= 63.7:
        return 'G'    
    elif df['NX'] > 63.7 and df['NX'] <= 84.2:
        return 'H'    
    elif df['NX'] > 84.2 and df['NX'] <= 130.5:
        return 'I'    
    elif df['NX'] > 130.5:
        return 'J'    
    
NX_highest['Expression'] = NX_highest.apply(classifier_highest_NX, axis=1)

In [ ]:
#To check the groups
#Expression = NX_highest.groupby('Expression').count() 

In [ ]:
ax = sns.violinplot(x="Expression", y="THSA", data=NX_highest)

ax = sns.violinplot(x="Expression", y="RHSA", data=NX_highest)

ax = sns.violinplot(x="Expression", y="LHPSA", data=NX_highest)

# Remove Transmembrane proteins

In [ ]:
NX_highest_wtmp = NX_highest[~NX_highest['Uniprot_ID'].isin(TM_uniprot)]

In [ ]:
ax = sns.violinplot(x="Expression", y="THSA", data=NX_highest_wtmp)

ax = sns.violinplot(x="Expression", y="RHSA", data=NX_highest_wtmp)

ax = sns.violinplot(x="Expression", y="LHPSA", data=NX_highest_wtmp)

# Median

consensus['Median_NX'].describe()

In [ ]:
def classifier_median(df):
    if df['Median_NX'] <= 1.4:
        return 'Low'
    elif df['Median_NX'] > 1.4 and df['Median_NX'] <= 10.3:
        return 'Medium'
    elif df['Median_NX'] > 10.3:
        return 'High'
    
NX_median['Expression'] = NX_median.apply(classifier_median, axis=1)

In [ ]:
#Check the groups
#Expression = NX_median.groupby('Expression').count()

In [ ]:
ax = sns.violinplot(x="Expression", y="THSA", data=NX_median)

ax = sns.violinplot(x="Expression", y="RHSA", data=NX_median)

ax = sns.violinplot(x="Expression", y="LHPSA", data=NX_median)

# Analysing outliers, Check what kind of proteins are on top (high expression and high hydrophobicity)

In [ ]:
NX_median_wtmp = NX_median[~NX_median['Uniprot_ID'].isin(TM_uniprot)] #removing transmembrane proteins
outliers = NX_median_wtmp[NX_median_wtmp['Expression'] == 'High']
outliers = outliers[outliers['RHSA'] > 0.4]
outliers = outliers[outliers['LHPSA'] > 2000]
outliers = outliers[outliers['THSA'] > 1500]
outliers = outliers.sort_values('NX')

all_uniprot = my_path + '/uniprot_9606.tab' # need to import once again to have all the info
all_uniprot = pd.read_csv(all_uniprot, sep='\t', engine='python') #all information from Uniprot only
all_uniprot.columns = ['Uniprot_ID', 'x1', 'x2', 'x3', 'Gene_ID', 'Protein', 'x5', 'x6', 'length']
all_uniprot = all_uniprot[['Uniprot_ID', 'Protein']]
outliers = pd.merge(outliers, all_uniprot, on='Uniprot_ID')